In [118]:
import pandas as pd
from pyparsing import nestedExpr



problem = "((-p or q) imp (q and (p or r)))"
args = ['p','q','r']

def implication(a,b):
    if a == True and b == False:
        return False
    else:
        return True
    
negate = lambda a: not a
log_and = lambda a,b: a and b
log_or = lambda a,b: a or b
log_equal = lambda a,b: a==b

def generate_next_list(amount,a,args):
    res_dict = {}
    all_variants = pow(2,amount)
    start_divider = 4
    var_count = int(all_variants/start_divider)
    for n in range(amount-1):
        res_list = []
        cycle_count = int(all_variants/var_count)
        
        for k in range(cycle_count):
            if k % 2 == 0 :
                res_list += [True for num in range(var_count)]
            else:
                res_list += [False for num in range(var_count)]
        res_dict[args[n+1]] = res_list
        var_count = int(var_count/2)
        
    return res_dict


amount = len(args)
amount_pow = pow(2,amount)
amount_pow_half = int(amount_pow/2)
a = [True] * amount_pow_half + [False] * amount_pow_half




def bool_to_int(answers_dict):
    new_dict = {}
    for letter in answers_dict:
        new_list  = [int(value) for value in answers_dict[letter]]
        new_dict[letter] = new_list
    return new_dict

def replace_letters(input_list,letter,number):
    for i in range(len(input_list)):
        if isinstance(input_list[i], list):
            replace_letters(input_list[i],letter,number)
        elif isinstance(input_list[i],str) and input_list[i] == (f'-{letter}'):
            negated = int(negate(number))
            input_list[i] = negated
        elif isinstance(input_list[i], str) and input_list[i] == letter:
            input_list[i] = number
# Display the DataFrame

result_dict = {args[0]:a}
generated = generate_next_list(amount,a,args)
result_dict.update(generated)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
result_dict_new = bool_to_int(result_dict)

df1 = pd.DataFrame(result_dict_new)
print(df1)
    
def is_one_dimensional(input_list):
    for item in input_list:
        if isinstance(item, list):
            return False
    return True

def solve(expr_list):
    
    
    if len(expr_list) >1:
        
        
        if type(expr_list[0])==str and '-' in  expr_list[0]:
            expr_list[0] = negate(expr_list[0])
        if type(expr_list[2])==str and'-' in expr_list[2]:
            expr_list[2] = negate(expr_list[2])
    
        for index,item in enumerate(expr_list):
            if type(item) == str and item.isdigit():
                expr_list[index] = int(item)
        match expr_list[1]:

            case 'or'|"o1"|"o0":
                answer = log_or(expr_list[0],expr_list[2])
                
            case 'and':
                answer = log_and(expr_list[0],expr_list[2])
               
            case 'imp'|"im1"|"im0":
                answer = implication(expr_list[0],expr_list[2])
               
            case 'equal'|"e1ual"|"e0qual":
                answer = log_equal(expr_list[0],expr_list[2])
                
        
        expr_list = answer
        
        return expr_list

def process_nested_lists(nested_list):
    if isinstance(nested_list, list):
        for i,item in enumerate(nested_list):
            check_var = process_nested_lists(item)
            if check_var is not None:
                nested_list[i] = process_nested_lists(item)
        return solve(nested_list)



def solve_for_all(problem,result_dict):
    max_length = max(len(lst) for lst in result_dict.values())
    keys = result_dict.keys()
    print(keys,'keys')
    results_list = []
    for i in range(max_length):
        values = []
        

        for key, value_list in result_dict.items():
            if i < len(value_list):
                print(value_list[i],"value list curr value")
                values.append(value_list[i])
         
        
        nested_list = nestedExpr().parseString(problem).asList()
        nested_list = nested_list[0]
        print(nested_list,'iyo')
        for i,key in enumerate(keys):
            replace_letters(nested_list,key,values[i])
            print(key,values[i], 'eseni gadaeca')
        print(nested_list,'gaxda')
        
        processed = process_nested_lists(nested_list)
        
        
        results_list.append(processed)
    return results_list
        

result_dict_new['res'] = solve_for_all(problem,result_dict_new)
print(result_dict_new)

df2 = pd.DataFrame(result_dict_new)
print(df2)

   p  q  r
0  1  1  1
1  1  1  0
2  1  0  1
3  1  0  0
4  0  1  1
5  0  1  0
6  0  0  1
7  0  0  0
dict_keys(['p', 'q', 'r']) keys
1 value list curr value
1 value list curr value
1 value list curr value
[['-p', 'or', 'q'], 'imp', ['q', 'and', ['p', 'or', 'r']]] iyo
p 1 eseni gadaeca
q 1 eseni gadaeca
r 1 eseni gadaeca
[[0, 'or', 1], 'imp', [1, 'and', [1, 'or', 1]]] gaxda
1 value list curr value
1 value list curr value
0 value list curr value
[['-p', 'or', 'q'], 'imp', ['q', 'and', ['p', 'or', 'r']]] iyo
p 1 eseni gadaeca
q 1 eseni gadaeca
r 0 eseni gadaeca
[[0, 'or', 1], 'imp', [1, 'and', [1, 'or', 0]]] gaxda
1 value list curr value
0 value list curr value
1 value list curr value
[['-p', 'or', 'q'], 'imp', ['q', 'and', ['p', 'or', 'r']]] iyo
p 1 eseni gadaeca
q 0 eseni gadaeca
r 1 eseni gadaeca
[[0, 'or', 0], 'imp', [0, 'and', [1, 'or', 1]]] gaxda
1 value list curr value
0 value list curr value
0 value list curr value
[['-p', 'or', 'q'], 'imp', ['q', 'and', ['p', 'or', 'r']]] iyo
p 1 e

In [101]:
import pandas as pd
from pyparsing import nestedExpr



problem = "((-1 or 1) imp (0 and (1 or 1)))"
args = ['p','q','r']

def implication(a,b):
    if a == True and b == False:
        return False
    else:
        return True
    
negate = lambda a: not a
log_and = lambda a,b: a and b
log_or = lambda a,b: a or b
log_equal = lambda a,b: a==b

def is_one_dimensional(input_list):
    for item in input_list:
        if isinstance(item, list):
            return False
    return True



def solve(expr_list):
    
    
    if len(expr_list) >1:
        
        
        if type(expr_list[0])==str and '-' in  expr_list[0]:
            expr_list[0] = negate(expr_list[0])
        if type(expr_list[2])==str and'-' in expr_list[2]:
            expr_list[2] = negate(expr_list[2])
    
        for index,item in enumerate(expr_list):
            if type(item) == str and item.isdigit():
                expr_list[index] = int(item)
        match expr_list[1]:

            case 'or'|"o1"|"o0":
                answer = log_or(expr_list[0],expr_list[2])
                
            case 'and':
                answer = log_and(expr_list[0],expr_list[2])
               
            case 'imp'|"im1"|"im0":
                answer = implication(expr_list[0],expr_list[2])
               
            case 'equal'|"e1ual"|"e0qual":
                answer = log_equal(expr_list[0],expr_list[2])
                
        
        expr_list = answer
        
        return expr_list

def process_nested_lists(nested_list):
    if isinstance(nested_list, list):
        
        for i,item in enumerate(nested_list):
            print(nested_list)
            check_var = process_nested_lists(item)
            if check_var is not None:
                nested_list[i] = process_nested_lists(item)
                
        print(solve(nested_list),'esaa pasuxi')
        
nested_list = nestedExpr().parseString(problem).asList()
process_nested_lists(nested_list)

[[['-1', 'or', '1'], 'imp', ['0', 'and', ['1', 'or', '1']]]]
[['-1', 'or', '1'], 'imp', ['0', 'and', ['1', 'or', '1']]]
['-1', 'or', '1']
['-1', 'or', '1']
['-1', 'or', '1']
1 esaa pasuxi
[[False, 'or', 1], 'imp', ['0', 'and', ['1', 'or', '1']]]
[[False, 'or', 1], 'imp', ['0', 'and', ['1', 'or', '1']]]
['0', 'and', ['1', 'or', '1']]
['0', 'and', ['1', 'or', '1']]
['0', 'and', ['1', 'or', '1']]
['1', 'or', '1']
['1', 'or', '1']
['1', 'or', '1']
1 esaa pasuxi
0 esaa pasuxi
True esaa pasuxi
None esaa pasuxi


In [6]:
import re
from pyparsing import nestedExpr

def is_one_dimensional(input_list):
    for item in input_list:
        if isinstance(item, list):
            return False
    return True

def parse(expr_list):
    if not is_one_dimensional(expr_list):
        pass
        

def solve(expr_list):
    if is_one_dimensional(expr_list):
        
        if '-' in  expr_list[0]:
            expr_list[0] = negate(expr_list[0])
        if '-' in expr_list[2]:
            expr_list[2] = negate(expr_list[2])
        match expr_list[1]:
            
            case 'or':
                answer = log_or(expr_list[0],expr_list[2])
            case 'and':
                answer = log_and(expr_list[0],expr_list[2])
            case 'imp':
                answer = implication(expr_list[0],expr_list[2])
            case 'equal':
                answer = log_equal(expr_list[0],expr_list[2])
        expr_list.clear()
        expr_list.append(answer)
        




print (expr_as_list)
expr_string = ""
bin_1 = ''
operator = ''
bin_2 = ''
for expr in expr_as_list:
    bin_1 = expr[0]
    operator = expr[1]
    bin_2 = expr[2]
    print(bin_1)
    print(operator)
    print(bin_2)
     


[[['-p', 'or', 'q'], 'imp', ['q', 'and', ['p', 'or', 'r']]]]
['-p', 'or', 'q']
imp
['q', 'and', ['p', 'or', 'r']]


In [102]:
def is_one_dimensional(input_list):
    for item in input_list:
        if isinstance(item, list):
            return False
    return True

def solve(expr_list):
    
    
    if len(expr_list) >1:
        print(expr_list)
        
        if type(expr_list[0])==str and '-' in  expr_list[0]:
            expr_list[0] = negate(expr_list[0])
        if type(expr_list[2])==str and'-' in expr_list[2]:
            expr_list[2] = negate(expr_list[2])
    
        for index,item in enumerate(expr_list):
            if type(item) == str and item.isdigit():
                expr_list[index] = int(item)
        match expr_list[1]:

            case 'or':
                answer = log_or(expr_list[0],expr_list[2])
                print('or')
            case 'and':
                answer = log_and(expr_list[0],expr_list[2])
                print('and')
            case 'imp':
                answer = implication(expr_list[0],expr_list[2])
                print('imp')
            case 'equal':
                answer = log_equal(expr_list[0],expr_list[2])
                print('equal')
        
        expr_list = answer
        print(f"answer is {expr_list} ")
        return expr_list

def process_nested_lists(nested_list):
    if isinstance(nested_list, list):
        print(nested_list,' process')
        for i,item in enumerate(nested_list):
            check_var = process_nested_lists(item)
            if check_var is not None:
                nested_list[i] = process_nested_lists(item)
        return solve(nested_list)


# Example usage

nested_list = [['0', 'or', '0'], 'imp', [['1', 'and', '1'], 'and', ['1', 'or', '0']]]
process_nested_lists(nested_list)

[['0', 'or', '0'], 'imp', [['1', 'and', '1'], 'and', ['1', 'or', '0']]]  process
['0', 'or', '0']  process
['0', 'or', '0']
or
answer is 0 
[0, 'or', 0]  process
[0, 'or', 0]
or
answer is 0 
[['1', 'and', '1'], 'and', ['1', 'or', '0']]  process
['1', 'and', '1']  process
['1', 'and', '1']
and
answer is 1 
[1, 'and', 1]  process
[1, 'and', 1]
and
answer is 1 
['1', 'or', '0']  process
['1', 'or', '0']
or
answer is 1 
[1, 'or', 0]  process
[1, 'or', 0]
or
answer is 1 
[1, 'and', 1]
and
answer is 1 
[1, 'and', 1]  process
[1, 'and', 1]
and
answer is 1 
[0, 'imp', 1]
imp
answer is True 


True

In [4]:
import re
problem = "(p or q) imp (q and (p or r))"
def solve(problem_string):
    main_problem_sign_pattern = r'\)\s+([^()]+)\s+\('
    two_problems_patterns = r'\([^)]+\s+(?:or|and|imp)\s+[^)]+\)+'
    extract_main_problem_sign = re.findall(main_problem_sign_pattern,problem_string)
    exctract_two_problems = re.findall(two_problems_patterns,problem_string)
    return exctract_two_problems
print(solve(problem))
    

['(p or q)', '(q and (p or r))']


In [38]:
import regex as re

problem = "(p or q) imp (q and (p or r))"
two_problems_patterns = r'\(([^()]|(?R))*\)'
extract_two_problems = re.findall(two_problems_patterns, problem)
exctract_two_problems

print(extract_two_problems)


['q', '(p or r)']


In [93]:
def is_one_dimensional(input_list):
    for item in input_list:
        if isinstance(item, list):
            return False
    return True

def solve(expr_list):
    
    if len(expr_list) >1:
    
        if '-' in  expr_list[0]:
            expr_list[0] = negate(expr_list[0])
        if '-' in expr_list[2]:
            expr_list[2] = negate(expr_list[2])
        match expr_list[1]:

            case 'or':
                answer = log_or(expr_list[0],expr_list[2])
                print('or')
            case 'and':
                answer = log_and(expr_list[0],expr_list[2])
                print('and')
            case 'imp':
                answer = implication(expr_list[0],expr_list[2])
                print('imp')
            case 'equal':
                answer = log_equal(expr_list[0],expr_list[2])
                print('equal')
        expr_list.clear()
        expr_list = answer

def process_nested_lists(nested_list):
    if isinstance(nested_list, list):
        print(nested_list)
        for item in nested_list:
            process_nested_lists(item)
        
        solve(nested_list)
        print(nested_list, end=' ')
        print('solved')


# Example usage

nested_list = [['0', 'or', '0'], 'and', ['0', 'and', ['1', 'or', '0']]]
process_nested_lists(nested_list)

[['0', 'or', '0'], 'and', ['0', 'and', ['1', 'or', '0']]]
['0', 'or', '0']
or
[] solved
['0', 'and', ['1', 'or', '0']]
['1', 'or', '0']
or
[] solved
and
[] solved
and
[] solved


In [4]:
import re

text = "(apple banana) or (cherry date)"
pattern = r'\([^)]+\s+(?:or|and|imp)\s+[^)]+\)'

matches = re.findall(pattern, text)

for match in matches:
    print("Match:", match)
print(matches)

Match: (apple or banana)
['(apple or banana)']


In [130]:
import pandas as pd

def generate_next_list(amount,a,args):
    res_dict = {}
    all_variants = pow(2,amount)
    start_divider = 4
    var_count = int(all_variants/start_divider)
    for n in range(amount-1):
        res_list = []
        cycle_count = int(all_variants/var_count)
        
        for k in range(cycle_count):
            if k % 2 == 0 :
                res_list += [True for num in range(var_count)]
            else:
                res_list += [False for num in range(var_count)]
        res_dict[args[n+1]] = res_list
        var_count = int(var_count/2)
        
    return res_dict

args = ['p','q','r','t']
amount = len(args)
amount_pow = pow(2,amount)
amount_pow_half = int(amount_pow/2)
a = [True] * amount_pow_half + [False] * amount_pow_half


def bool_to_int(answers_dict):
    new_dict = {}
    for letter in answers_dict:
        new_list  = [int(value) for value in answers_dict[letter]]
        new_dict[letter] = new_list
    return new_dict
# Display the DataFrame

result_dict = {args[0]:a}
generated = generate_next_list(amount,a,args)
result_dict.update(generated)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
result_dict_new = bool_to_int(result_dict)

df1 = pd.DataFrame(result_dict_new)
print(df1)

    p  q  r  t
0   1  1  1  1
1   1  1  1  0
2   1  1  0  1
3   1  1  0  0
4   1  0  1  1
5   1  0  1  0
6   1  0  0  1
7   1  0  0  0
8   0  1  1  1
9   0  1  1  0
10  0  1  0  1
11  0  1  0  0
12  0  0  1  1
13  0  0  1  0
14  0  0  0  1
15  0  0  0  0


In [111]:
dict1 = {'a': 1, 'b': 2}
dict2 = {'k': 3, 'c': 4}


for t in dict1:
    print(dict1[t])


1
2


In [40]:
expr_list_1 = ['1', 'or', '0']
for index,item in enumerate(expr_list_1):
    if item.isdigit():
        expr_list_1[index] = int(item)
print(expr_list_1)

[1, 'or', 0]


In [27]:
import tkinter as tk

def on_button_click():
    label.config(text="Button Clicked!")

def display_text():
    text_content = text_field.get("1.0", "end-1c")
    print("Text entered:", text_content)

root = tk.Tk()
root.title("Simple GUI")
root.geometry('800x600')

label = tk.Label(root, text="Hello, GUI!")
label.pack()
# Create a label
label = tk.Label(root, text="Enter Text:")
label.pack()

text_field = tk.Text(root, height=5, width=40)  # Adjust height and width as needed
text_field.pack()

button = tk.Button(root, text="Click Me", command=display_text)
button.pack()

root.mainloop()


In [108]:
values = []
result_dict = {"p":[1,2,3], "q":[1,2,3,4]}
result_dict['a'] = []

for key, value_list in result_dict.items():
    if i < len(value_list):
        
        values.append(value_list[i])

    else:
        values.append(None)
print(values)

p
q
[3, 3, None]


In [113]:
a = 1
print(int(negate(1)))

0
